In [25]:
### Library Imports
import pandas as pd
import yfinance as yf
import numpy as np
from scipy import stats
import datetime
import matplotlib.pyplot as plt


In [26]:
### Function to Import Stock Data
def import_stock_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date, end_date)['Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date, end_date)['Close']
    
    # Reset index to include the Date as a column
    data = data.reset_index()

    return data

tickers = ['AAPL', 'MSFT']
start_date = '2025-01-01'
end_date = '2025-06-01'
stock_data = import_stock_data(tickers, start_date, end_date)
print(stock_data.head())


/var/folders/n5/bknqp6qs37l5l4bwv9wv3bs40000gn/T/ipykernel_53809/17277343.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[tickers] = yf.download(tickers, start_date, end_date)['Close']
[*********************100%***********************]  2 of 2 completed

        Date        AAPL        MSFT
0 2025-01-02  243.263199  416.976868
1 2025-01-03  242.774368  421.728607
2 2025-01-06  244.410416  426.211365
3 2025-01-07  241.627136  420.752350
4 2025-01-08  242.115952  422.933960


In [27]:
### Compute the Percentage Change
def pct_change(tickers, df):
    # Add col with daily pct change for each ticker
    for t in tickers:
        df['PCT ' + t] = df[t].pct_change()

    return df

### Function Call to Compute the Percentage Change
stock_data = pct_change(tickers, stock_data)
print(stock_data.tail())


          Date        AAPL        MSFT  PCT AAPL  PCT MSFT
97  2025-05-23  195.270004  450.179993 -0.030244 -0.010289
98  2025-05-27  200.210007  460.690002  0.025298  0.023346
99  2025-05-28  200.419998  457.359985  0.001049 -0.007228
100 2025-05-29  199.949997  458.679993 -0.002345  0.002886
101 2025-05-30  200.850006  460.359985  0.004501  0.003663


In [ ]:
### Generate Buy Signals
''' 
Generate a "BUY" signal if the stock loses more than 5% of its value in a single day
'''
def generate_signals(df, drop_threshold):
    """
    Generate 'BUY' signals if a stock drops more than the threshold in a day.
    """
    for t in tickers:
        signal_col = 'Signal ' + t
        pct_col = 'PCT ' + t
        df[signal_col] = (df[pct_col] < drop_threshold).astype(int)
        # Test to see the number of generated signals
        print(f"{signal_col}: {df[signal_col].sum()} buy signals generated")

    return df

### Function Retun to Generate Buy Signals
stock_data = generate_signals(stock_data, drop_threshold = -0.02)
print(stock_data.head(10))


Signal AAPL: 20 buy signals generated
Signal MSFT: 10 buy signals generated
        Date        AAPL        MSFT  PCT AAPL  PCT MSFT  Signal AAPL  \
0 2025-01-02  243.263199  416.976868       NaN       NaN            0   
1 2025-01-03  242.774368  421.728607 -0.002009  0.011396            0   
2 2025-01-06  244.410416  426.211365  0.006739  0.010629            0   
3 2025-01-07  241.627136  420.752350 -0.011388 -0.012808            0   
4 2025-01-08  242.115952  422.933960  0.002023  0.005185            0   
5 2025-01-10  236.280045  417.345459 -0.024104 -0.013214            1   
6 2025-01-13  233.835922  415.592194 -0.010344 -0.004201            0   
7 2025-01-14  232.718613  414.078033 -0.004778 -0.003643            0   
8 2025-01-15  237.297562  424.677246  0.019676  0.025597            0   
9 2025-01-16  227.710693  422.953857 -0.040400 -0.004058            1   

   Signal MSFT  
0            0  
1            0  
2            0  
3            0  
4            0  
5            0  
6